In [1]:
import tensorflow as tf

from tensorflow.keras import layers, callbacks, models, utils

rooter = "PetImages"

In [ ]:
from tensorflow.keras.utils import image_dataset_from_directory

subset = "both"; label = "binary"

image = (120, 120); split = 0.1; random = 128

unseet = 10

orifile = image_dataset_from_directory(
  rooter,
  seed = unseet,
  subset = subset,
  image_size = image,
  label_mode = label,
  batch_size = random,
  validation_split = split
)

train, test = orifile

In [3]:
augmentation_model = models.Sequential([ 
  layers.RandomRotation(0.2, input_shape=(120, 120, 3)),
  layers.RandomZoom(0.2),
  layers.RandomFlip()
])

In [4]:
model = models.Sequential([ 
  layers.Rescaling(1.0 / 255.0, input_shape=(120, 120, 3)),
  augmentation_model,
  layers.Conv2D(16, 3, activation="relu", padding="same"),
  layers.MaxPool2D((2, 2)),
  layers.Conv2D(32, 3, activation="relu", padding="same"),
  layers.MaxPool2D((2, 2)),
  layers.Conv2D(64, 3, activation="relu", padding="same"),
  layers.MaxPool2D((2, 2)),
  layers.Conv2D(128, 3, activation="relu", padding="same"),
  layers.MaxPool2D((2, 2)),
  layers.Flatten(),
  layers.Dense(128, activation="relu"),
  layers.Dense(1, activation="sigmoid")
])

loss = "binary_crossentropy"

model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])

In [5]:
class EarlyStoppingCallback(callbacks.Callback):
  def on_epoch_end(self, epochs, logs={}):
    if logs.get("accuracy") >= 0.8 and logs.get("val_accuracy") >= 0.8:
      self.model.stop_training = True

callbacks = EarlyStoppingCallback()

In [ ]:
import warnings

warnings.filterwarnings("ignore")

history = model.fit(train, epochs=35, validation_data=test, callbacks=callbacks)

In [7]:
model.evaluate(train, verbose=False)

[0.3721517026424408, 0.8299737572669983]

In [8]:
model.evaluate(test, verbose=False)

[0.4058811366558075, 0.8091841340065002]

In [9]:
total_loop = len(history.history["accuracy"])

total_loop

15